# Discovery: how to use Data API to query RedShift from Python

## Setup: configure your cluster access, your role ARN, your output S3 URI

In the following cell, you will be prompted to provide some of the information associated to your Redshift cluster and S3 path of output.

In [ ]:
import sagemaker
session = sagemaker.Session()

###### CLUSTER CONFIGURATION
cluster_id = input("The name of your Redshift cluster:")
database = input("The database of your Redshift cluster (default: dev)") or 'dev'
db_user = input("The user of your Redshift cluster (default: awsuser)") or 'awsuser'

Now that Redshift cluster is set, we can now prepare our SQL query string. In this example, we plan on predicting total sales for a specific event, provided its venue, category, date and holiday information. The query is a pretty basic one but can be improved as needed.

In [ ]:
###### QUERY STRING
query_string = """
-- Find total sales for specific event, plus additional features
SELECT sum(s.qtysold) AS total_sold, sum(s.pricepaid) AS total_paid, e.venueid, e.catid, d.caldate, d.holiday
from sales s, event e, date d
WHERE s.eventid = e.eventid and e.dateid = d.dateid
GROUP BY e.venueid, e.catid, d.caldate, d.holiday
""" # this will work on the default Free Tier Redshift cluster. Change if needed.

You can now use the AWS SDK for Python, `boto3`, to query the Redshift Data API. You can check more details on the boto3 client at this [link](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift-data.html#client).

In [ ]:
import boto3
import time

# Execute the Data API query
client = boto3.client('redshift-data')
execution_id = client.execute_statement(
    ClusterIdentifier=cluster_id,
    Database=database,
    DbUser=db_user,
    Sql=query_string,
)['Id']
print(f'Execution started with ID {execution_id}')

In [ ]:
# Wait for the query to be done
status = client.describe_statement(Id=execution_id)['Status']
while status not in ['FINISHED','ABORTED','FAILED']:
    time.sleep(10)
    status = client.describe_statement(Id=execution_id)['Status']
print(f'Execution {execution_id} finished with status {status}')

In [ ]:
if status == 'FINISHED':
    columns = [c['label'] for c in client.get_statement_result(Id=execution_id)['ColumnMetadata']]
    records = client.get_statement_result(Id=execution_id)['Records']
    print(f'SUCCESS. Found {len(records)} records')
else:
    print(f'Failed with Error: {client.describe_statement(Id=execution_id)["Error"]}')

Once the query return successful, we can now parse the content and use it to create a Pandas DataFrame.

In [ ]:
from io import StringIO

def content_parser(columns, records):
    # Build the CSV
    content = ""
    line = ""
    for c in columns:
        line+=f"{c},"
    content += line[:-1]+'\n'
    for record in records:
        line = ""
        for v in record:
            data = v.get('longValue') or v.get('stringValue')
            line += f'{data},'
        content += line[:-1]+'\n'
    return StringIO(content)

data = content_parser(columns, records)

In [ ]:
import pandas as pd

df = pd.read_csv(data, sep=',')
df.head()